## PyPSA Introduction Notebook

### 🎯 Objective

This notebook provides a detailed introduction to the `pypsa.Network()` object. It covers:

- The components that make up PyPSA’s graph-based network model.
- Key features for exploring and inspecting the network.
- Methods for updating components and applying changes.  
- Tools for viewing constraints and solving the optimization problem.  


### ✍️ Authors
<div style="display: flex; align-items: center; justify-content: space-between; flex-wrap: nowrap; gap: 2rem;">

  <div style="flex: 1; min-width: 250px;">
      <a href="https://www.linkedin.com/in/virio-andreyana" target="_blank">Virio Andreyana</a><br>
      <a href="https://www.linkedin.com/in/andreas-denyer" target="_blank">Andreas Denyer</a><br>
      <a href="https://www.linkedin.com/in/priyeshgosai" target="_blank">Priyesh Gosai</a>
    </p>
  </div>

  <div style="flex-shrink: 0;">
    <a href="https://openenergytransition.org/index.html" target="_blank">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="Open Energy Transition logo">
    </a>
  </div>

</div>


### ⚙️ Setup Environment

In [ ]:
# @title Install the required packages
# @markdown Run this cell to install the necessary Python packages for the project.
import subprocess, sys, importlib

try:
    from importlib import metadata  # Python 3.8+
except ImportError:
    import importlib_metadata as metadata  # Backport for older versions

packages = [
    "pypsa",        # power-system modelling & optimization toolbox
    "pypsa[excel]", # pypsa with Excel I/O support
    "folium",       # interactive leaflet-based maps in Python
    "mapclassify"   # spatial data classification for choropleth maps
]

# Install packages
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", *packages],
    check=True
)

# Check and report installed versions
base_names = [pkg.split("[")[0] for pkg in packages]
missing = [p for p in base_names if importlib.util.find_spec(p) is None]

if not missing:
    print("✅ All packages installed successfully.\n")
    for pkg in base_names:
        try:
            version = metadata.version(pkg)
            print(f"📦 {pkg} version {version}")
        except metadata.PackageNotFoundError:
            print(f"⚠️ {pkg} is not found in metadata.")
else:
    print(f"❌ Missing: {', '.join(missing)}")



### 🧭 Navigating the PyPSA Object

**Initialize the PyPSA Network** <br>
This cell:
* imports the necessary libraries
* changes the plotting tool from the default `matplotlib` to the interactive `plotly` library. 
* initializes a PyPSA network  using the meshed AC/DC example.

```python
import <package>
```

or in the case you want to make an shorthand for a package 

```python
import <package> as <shorthand>
```

In [ ]:
import pypsa
import pandas as pd
pd.options.plotting.backend = "plotly"
network = pypsa.examples.ac_dc_meshed()

To create a blank PyPSA `Network`, use:

```python
network = pypsa.Network()
```

To import an existing network from a supported file format (e.g. HDF5, NetCDF, CSV folder, or Excel), pass the file path:

```python
network = pypsa.Network("filename.xx")
```


Display Network Components

In [ ]:
network.components

We use a `for` loop to illustrate PyPSA's naming conventions: 
* `PascalCase` is used for component classes. 
* `snake_case` is used for attributes and methods.

In [ ]:
for key in network.component_attrs:
    print(f'{key.ljust(20)} {network.components[key]["list_name"]}')

**Component Attributes**


Change the `ComponentName` using PascalCase as a string to view the attributes. 


In [ ]:
ComponentName = 'Generator' # or use 'Bus''Link','Line','Carrier' for other components.

network.component_attrs[ComponentName]

**Methods in the Network Object**

We use a for loop to display all available methods in the pypsa.Network object. 

These methods allow you to interact with and modify the network.

_For better readability, the output is arranged in a grid format._

In [ ]:
import inspect

methods = sorted(
    name for name, func in inspect.getmembers(network, inspect.ismethod)
    if not name.startswith("_")
)

n_cols = 3
n_rows = (len(methods) + n_cols - 1) // n_cols
col_width = max(len(m) for m in methods) + 2

for row_idx in range(n_rows):
    for col_idx in range(n_cols):
        idx = col_idx * n_rows + row_idx
        if idx < len(methods):
            print(methods[idx].ljust(col_width), end="")
    print()

ℹ️ **Where to Learn More**  

There's a lot of information in this notebook, and users are encouraged to explore the [PyPSA documentation](https://pypsa.readthedocs.io/).  
It offers comprehensive details on components, methods, API usage, and the latest release notes.



**View the Signature, Docstring, and Source Location**

PyPSA follows well-established coding standards, making it easy to explore and understand the codebase.  
To inspect a method’s signature, documentation string, and source location, use:

```python
network.<method>?
```

This will display helpful information in the output block, including the method's arguments and a short description of its functionality.

In [ ]:
network.add?

**Explore the Network**

This cell provides an interactive map of the network using the `explore` method using Folium.


In [ ]:
network.plot.explore()

**View Input Data**

To view input data in PyPSA, you can access the corresponding component using its **snake_case** name directly on the `network` object.



In [ ]:
network.buses

In [ ]:
network.lines

In [ ]:
network.links

In [ ]:
network.loads

In [ ]:
network.generators

**Static vs Time-Varying Data**

PyPSA distinguishes between:

- **Static input data**, which does not change over time (e.g. `network.generators`, `network.loads`)
- **Time-varying data**, which changes across snapshots and is accessed via the `_t` suffix

The time dimension is defined by the `network.snapshots` attribute, which specifies the set of timestamps over which the model is solved.

In [ ]:
network.snapshots

In [ ]:
network.loads_t.p_set

In [ ]:
network.generators_t.p_max_pu

**Constraints in PyPSA**

Within PyPSA, all system data is formulated into mathematical constraints that define the optimization problem. These constraints are built using the `linopy` toolbox, which is also developed and maintained by the PyPSA team.

This design allows you to **construct and inspect the optimization model without actually solving it**.  
In other words, you can prepare and review the constraints independently before running the full optimization.

This is especially useful for debugging, understanding the model structure, or applying custom modifications.

>_**Adding Custom Constraints**_

>_In many applications, there may be constraints that are not generated by default in the PyPSA network.  
In these cases, you can **add custom constraints** to the model before solving the optimization problem._

>_An example of how to do this is available in the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/advanced_linear_constraints.html)._



In [ ]:
network.optimize.create_model()

We can view the constraints.

In [ ]:
network.model

**Optimize the Network**

This cell runs the optimization on the network. The default solver is HiGHS, which is a high-performance open-source solver for linear programming problems.

You can change the solver by modifying the `solver_name` parameter in the `network.optimize()` method.

Additional advanced features in PyPSA may be of interest to some users, including support for [custom constraints](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#custom-constraints), [rolling horizon optimization](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#rolling-horizon-optimization), and several other modeling options.

For a full overview, visit the [System Optimization](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#) section of the PyPSA documentation.


In [ ]:
network.optimize()

**Observe the Results**

Just as with input data, PyPSA provides access to both **static** and **time-varying** results after solving the model.

- **Static results** (e.g. capacities, marginal costs) are stored in attributes like `network.generators`.
- **Time-varying results** (e.g. dispatch, flows, state of charge) are stored using the `_t` suffix:


In [ ]:
network.objective

In [ ]:
network.generators_t.p       # Generator dispatch over time

In [ ]:
network.storage_units_t.p    # Storage power over time


In [ ]:
network.links_t.p0           # Link flows at the sending end

In the next section we look at the inputs and outputs for this network.

---